In [ ]:
!pip install statsbombpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
# Download the data ----

from statsbombpy import sb
import pandas as pd

# Remind ourselves of the competition ID and season ID for EPL 2015-16
competitions = sb.competitions()
competitions.loc[lambda x: x['country_name'] == 'England']

matches = sb.matches(competition_id=2, season_id=27)

# Use list comprehension to download event data for 100 matches (~ 2 seconds per match)
columns = [
    'match_id', 'index', 'team_id', 'player_id', 'player', 'location', 'type',
    'pass_end_location', 'carry_end_location', 'shot_end_location', 'goalkeeper_end_location',
    'pass_outcome', 'duel_outcome', 'dribble_outcome', 'goalkeeper_outcome', 'shot_outcome', 'interception_outcome',
    'pass_type', 'duel_type', 'goalkeeper_type'
]
events = pd.concat([sb.events(match_id=x)[columns] for x in matches['match_id']])

/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsbombpy/api_client.py:21: 

In [ ]:
# Construct the SPADL representation ----

import numpy as np

events = (
    events
        # By default, events are not sorted by index, and this is a requirement for us
        .sort_values(['match_id', 'index'])
        .assign(
#            start_time=lambda x: [60 * int(y[3:5]) + float(y[6:]) for y in x['timestamp']],
#            end_time=lambda x: x['start_time'] + np.where(x['duration'].isna(), 0, x['duration']),
            start_loc_x=lambda x: x['location'].apply(lambda y: y[0] if isinstance(y, list) else np.nan),
            start_loc_y=lambda x: x['location'].apply(lambda y: y[1] if isinstance(y, list) else np.nan),
            end_location=lambda x: np.select(
                condlist=[x['type'] == 'Pass', x['type'] == 'Carry', x['type'] == 'Shot', x['type'] == 'Goalkeeper'],
                choicelist=[x['pass_end_location'], x['carry_end_location'], x['shot_end_location'], x['goalkeeper_end_location']],
                default=x['location'],
            ),
            end_loc_x=lambda x: x['end_location'].apply(lambda y: y[0] if isinstance(y, list) else np.nan),
            end_loc_y=lambda x: x['end_location'].apply(lambda y: y[1] if isinstance(y, list) else np.nan),
#            body_part=lambda x: np.select(
#                condlist=[x['type'] == 'Pass', x['type'] == 'Shot', x['type'] == 'Clearance', x['type'] == 'Goalkeeper'],
#                choicelist=[x['pass_body_part'], x['shot_body_part'], x['clearance_body_part'], x['goalkeeper_body_part']],
#                default=None,
#            ),
            # Manually define the success/failure result for the most frequent events.
            # If we had more time, we'd want to handle ALL events.
            result=lambda x: np.select(
                condlist=[
                    x['type'] == 'Pass',
                    x['type'] == 'Carry',
                    x['type'] == 'Ball Recovery',
                    x['type'] == 'Duel',
                    x['type'] == 'Clearance',
                    x['type'] == 'Dribble',
                    x['type'] == 'Goalkeeper',
                    x['type'] == 'Dispossessed',
                    x['type'] == 'Miscontrol',
                    x['type'] == 'Shot',
                    x['type'] == 'Foul Committed',
                    x['type'] == 'Foul Won',
                    x['type'] == 'Dribbled Past',
                    x['type'] == 'Interception',
                ],
                choicelist=[
                    # Pass
                    np.select(
                        condlist=[x['pass_outcome'].isna(), x['pass_outcome'] == 'Incomplete', x['pass_outcome'] == 'Pass Offside'],
                        choicelist=['success', 'failure', 'offside'],
                        default=None
                    ),
                    'success', # Carry
                    'success', # Ball Recovery
                    # Duel
                    np.select(
                        condlist=[x['duel_type'] == 'Aerial Lost', x['duel_outcome'].isin(['Lost In Play', 'Lost Out']), x['duel_outcome'].isin(['Won', 'Success in Play'])],
                        choicelist=['failure', 'failure', 'success'],
                        default=None
                    ),
                    'success', # Clearance
                    np.where(x['dribble_outcome'] == 'Complete', 'success', 'failure'), # Dribble
                    # Goalkeeper
                    np.select(
                        condlist=[x['goalkeeper_type'].isin(['Shot Faced', 'Shot Saved', 'Collected']), x['goalkeeper_outcome'] == 'Claim'],
                        choicelist=['success', 'success'],
                        default=None
                    ),
                    'failure', # Dispossessed
                    'failure', # Miscontrol
                    np.where(x['shot_outcome'] == 'Goal', 'success', 'failure'),    # Shot
                    'failure', # Foul Committed
                    'success', # Foul Won
                    'success', # Dribbled Past
                    np.where(x['interception_outcome'] == 'Won', 'success', 'failure'), # Interception
                ],
                default=None
            ),
            # Create a new string that encodes both type and result
            type_result=lambda x: x['type'] + '_' + x['result'],
        )
        # Throw out any events for which the result is NA
        .loc[lambda x: ~x['result'].isna()]
)



In [ ]:
# Compute outcomes and features for the regression model ----

# This VAEP constant k defines the number of events we look into the future for goal events
k = 10

events = (
    events
        # Get the SPADL features for the two events that preceded the event in question
        .assign(
#            start_time_1=lambda x: x['start_time'].shift(1),
#            start_time_2=lambda x: x['start_time'].shift(2),
#            end_time_1=lambda x: x['end_time'].shift(1),
#            end_time_2=lambda x: x['end_time'].shift(2),
#            start_loc_x_1=lambda x: x['start_loc_x'].shift(1),
            start_loc_x_2=lambda x: x['start_loc_x'].shift(2),
#            start_loc_y_1=lambda x: x['start_loc_y'].shift(1),
            start_loc_y_2=lambda x: x['start_loc_y'].shift(2),
#            end_loc_x_1=lambda x: x['end_loc_x'].shift(1),
#            end_loc_x_2=lambda x: x['end_loc_x'].shift(2),
#            end_loc_y_1=lambda x: x['end_loc_y'].shift(1),
#            end_loc_y_2=lambda x: x['end_loc_y'].shift(2),
#            player_id_1=lambda x: x['player_id'].shift(1),
#            player_id_2=lambda x: x['player_id'].shift(2),
#            team_id_1=lambda x: x['team_id'].shift(1),
#            team_id_2=lambda x: x['team_id'].shift(2),
#            type_1=lambda x: x['type'].shift(1),
#            type_2=lambda x: x['type'].shift(2),
#            body_part_1=lambda x: x['body_part'].shift(1),
#            body_part_2=lambda x: x['body_part'].shift(2),
#            result_1=lambda x: x['result'].shift(1),
#            result_2=lambda x: x['result'].shift(2),
#            type_result_1=lambda x: x['type_result'].shift(1),
#            type_result_2=lambda x: x['type_result'].shift(2),
        )
        # Compute the outcome variables goals scored and goals conceded
        .assign(
            # Need to re-compute index after we've filtered down to SPADL events
            index_spadl=lambda x: range(x.shape[0]),
            # Set up some helper variables so that we can identify when the next goal happens.
            # First, grab the index, match ID and team ID for each goal (if no goal, this is NaN)
            goal_index_spadl=lambda x: np.where(x['shot_outcome'] == 'Goal', x['index_spadl'], np.nan),
            goal_match_id=lambda x: np.where(x['shot_outcome'] == 'Goal', x['match_id'], np.nan),
            goal_team_id=lambda x: np.where(x['shot_outcome'] == 'Goal', x['team_id'], np.nan),
            # Second, fill new columns upward to find the index, etc., of the next goal after each event
            next_goal_index_spadl=lambda x: x['goal_index_spadl'].fillna(method='bfill'),
            next_goal_match_id=lambda x: x['goal_match_id'].fillna(method='bfill'),
            next_goal_team_id=lambda x: x['goal_team_id'].fillna(method='bfill'),
            is_goal_scored=lambda x: list(
                map(    # Convert to int instead of logical so that probabilities below reflect success
                    int,
                    (x['next_goal_match_id'] == x['match_id']) &
                        (x['next_goal_team_id'] == x['team_id']) &
                        (x['next_goal_index_spadl'] <= x['index_spadl'] + 10)
                        # NOTE: I forgot to check that the period is the same
                )
            ),
            is_goal_conceded=lambda x: list(
                map(    # Convert to int instead of logical so that probabilities below reflect success
                    int,
                    (x['next_goal_match_id'] == x['match_id']) &
                        ~(x['next_goal_team_id'] == x['team_id']) &
                        (x['next_goal_index_spadl'] <= x['index_spadl'] + 10)
                        # NOTE: I forgot to check that the period is the same
                )
            ),
        )
        # Calculate predictive features for outcome models (far fewer than we saw in the VAEP paper)
        .assign(
            end_distance_opp_goal=lambda x: np.sqrt((x['end_loc_x']-120)**2 + (x['end_loc_y'] - 40)**2),
            end_distance_own_goal=lambda x: np.sqrt((x['end_loc_x'])**2 + (x['end_loc_y'] - 40)**2),
            downfield_movement=lambda x: x['end_loc_x'] - x['start_loc_x_2'],
        )
        .loc[lambda x: ~x['downfield_movement'].isna()]
)


In [ ]:
# Fit the logistic regression models ----

# We're fitting very simple models here, only using the action type, distance to goal and
# downfield movement over the past three actions.

import statsmodels.api as sm
import statsmodels.formula.api as smf

model_goal_scored = (
    smf.glm(
        formula="is_goal_scored ~ type_result + end_distance_opp_goal + downfield_movement",
        # Exclude goals from the outcome model because they result in a goal 100% of the time
        data=events.loc[lambda x: x['type_result'] != "Shot_success"],
        family=sm.families.Binomial()
    )
        .fit()
)

model_goal_conceded = (
    smf.glm(
        formula="is_goal_conceded ~ type_result + end_distance_own_goal + downfield_movement",
        data=events.loc[lambda x: x['type_result'] != "Shot_success"],
        family=sm.families.Binomial()
    )
        .fit()
)

model_goal_scored.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:         is_goal_scored   No. Observations:               805332
Model:                            GLM   Df Residuals:                   805312
Model Family:                Binomial   Df Model:                           19
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -39712.
Date:                Sun, 22 Oct 2023   Deviance:                       79424.
Time:                        02:48:39   Pearson chi2:                 1.18e+06
No. Iterations:                    11   Pseudo R-squ. (CS):            0.01192
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                -1.6920      0.052    -32.243      0.000      -1.795      -1.589
type_result[T.Carry_success]             -0.2583      0.049     -5.277      0.000      -0.354      -0.162
type_result[T.Clearance_success]         -0.2573      0.194     -1.330      0.184      -0.637       0.122
type_result[T.Dispossessed_failure]      -1.4689      0.145    -10.115      0.000      -1.753      -1.184
type_result[T.Dribble_failure]           -1.4948      0.189     -7.913      0.000      -1.865      -1.125
type_result[T.Dribble_success]            0.2747      0.085      3.250      0.001       0.109       0.440
type_result[T.Dribbled Past_success]     -2.0083      0.337     -5.955      0.000      -2.669      -1.347
type_result[T.Duel_failure]              -1.6403      0.125    -13.142      0.000      -1.885      -1.396
type_result[T.Duel_success]               0.0070      0.204      0.034      0.973      -0.393       0.407
type_result[T.Foul Committed_failure]    -3.3322      0.450     -7.406      0.000      -4.214      -2.450
type_result[T.Foul Won_success]           0.6575      0.086      7.627      0.000       0.489       0.826
type_result[T.Interception_failure]      -1.0040      0.338     -2.974      0.003      -1.666      -0.342
type_result[T.Interception_success]       0.1196      0.208      0.576      0.564      -0.287       0.526
type_result[T.Miscontrol_failure]        -1.8197      0.158    -11.517      0.000      -2.129      -1.510
type_result[T.Pass_failure]              -1.8940      0.068    -27.846      0.000      -2.027      -1.761
type_result[T.Pass_offside]              -3.4755      0.709     -4.900      0.000      -4.866      -2.085
type_result[T.Pass_success]              -0.2398      0.049     -4.924      0.000      -0.335      -0.144
type_result[T.Shot_failure]              -1.6997      0.088    -19.318      0.000      -1.872      -1.527
end_distance_opp_goal                    -0.0510      0.001    -76.494      0.000      -0.052      -0.050
downfield_movement                       -0.0005      0.000     -1.225      0.221      -0.001       0.000
=========================================================================================================
"""

In [ ]:
# Extract predictions from models ----

# We have to predict non-goal events separately from goal events because goal events
# were excluded from the training step.
events_no_goal = (
    events
        .loc[lambda x: x['type_result'] != 'Shot_success']
        .assign(
            pred_goal_scored=model_goal_scored.fittedvalues,
            pred_goal_conceded=model_goal_conceded.fittedvalues,
        )
)

events_goal = (
    events
        .loc[lambda x: x['type_result'] == 'Shot_success']
        .assign(
            pred_goal_scored=1,     # a goal event ALWAYS leads to a goal scoring
            # We'll assume that a goal event leads to a goal conceded in the next k events at the
            # same rate as non-goal shots (this is probably not a great assumption).
            pred_goal_conceded=events_no_goal.loc[lambda x: x['type'] == 'Shot']['pred_goal_conceded'].mean(),
        )
)

# Put goal event predictions and non-goal event predictions together and calculate the change
# in goal scored/conceded probabilities which each event.
events = (
    pd.concat([events_goal, events_no_goal])
        .sort_values(['match_id', 'index'])
        .assign(
            # We have to handle three different possibilities:
            # -     Kick off. To avoid debiting players for the change that happens between the
            #       goal event and the kick off, we force the VAEP value for this event to be zero.
            # -     The team ID is the same as the previous event. In this case, we subtract the
            #       goal scored probability of the previous event from the goal scored probability
            #       of the current event.
            # -     The team ID is different from the previous event. In this case, we subtract the
            #       goal CONCEDED probability of the previous event from the goal scored probability
            #       of the current event.
            diff_goal_scored=lambda x: np.select(
                condlist=[x['pass_type'] == 'Kick Off', x['team_id'] == x['team_id'].shift(1), ~(x['team_id'] == x['team_id'].shift(1))],
                choicelist=[0, x['pred_goal_scored'] - x['pred_goal_scored'].shift(1), x['pred_goal_scored'] - x['pred_goal_conceded']],
                default=None
            ),
            # For goal conceded probability, we follow similar logic to the above.
            diff_goal_conceded=lambda x: np.select(
                condlist=[x['pass_type'] == 'Kick Off', x['team_id'] == x['team_id'].shift(1), ~(x['team_id'] == x['team_id'].shift(1))],
                choicelist=[0, x['pred_goal_conceded'] - x['pred_goal_conceded'].shift(1), x['pred_goal_conceded'] - x['pred_goal_scored']],
                default=None
            ),
            # VAEP is defined as change in goal scored prob, minus change in goal conceded prob
            vaep=lambda x: x['diff_goal_scored'] - x['diff_goal_conceded']
        )
)

In [ ]:
# Aggregate VAEP by player ----

# First aggregate by player AND action type in case we want to look at the action-specific value
# for each player
player_type_summary = (
    events
        .groupby(['player_id', 'player', 'type'])
        .agg(
            count=('player_id', 'count'),
            vaep=('vaep', 'sum')
        )
        .reset_index()
)

# Now aggregate by player across action types
player_summary = (
    player_type_summary
        .groupby(['player_id', 'player'])
        .agg(
            count=('count', 'sum'),
            vaep=('vaep', 'sum')
        )
        .reset_index()
)

# Look at the top overall players according to VAEP
player_summary.sort_values('vaep', ascending=False)

,player_id,player,count,vaep
225,3814.0,Riyad Mahrez,3481,33.648504
404,10955.0,Harry Kane,3038,31.836809
407,10960.0,Jamie Vardy,2019,29.086459
35,3237.0,Sergio Leonel Agüero del Castillo,2359,28.49843
126,3491.0,Dimitri Payet,3560,25.370723
...,...,...,...,...
513,40123.0,Robert Huth,1801,-7.608537
49,3290.0,Federico Fernández,3637,-7.6423
511,40121.0,Philipp Wollscheid,2557,-8.174316
286,4746.0,Younes Kaboul,1587,-8.250634
